# Анализ производетльности такси в Нью-Йорке

Конечно! Это один из датасетов NYC Taxi & Limousine Commission (TLC) - данные о поездках в желтых такси Нью-Йорка. Вот подробное описание каждого поля:

## Основная информация о поездке

**`vendorid`** - Идентификатор провайдера/вендора такси
- 1 = Creative Mobile Technologies (CMT)
- 2 = VeriFone Inc. (VTS)

**`tpep_pickup_datetime`** - Дата и время начала поездки (Timestamp)

**`tpep_dropoff_datetime`** - Дата и время окончания поездки (Timestamp)

**`passenger_count`** - Количество пассажиров в такси

## Географическая информация

**`trip_distance`** - Расстояние поездки в милях

**`pulocationid`** - ID зоны такси (Taxi Zone) для посадки
- Ссылается на таблицу taxi zones

**`dolocationid`** - ID зоны такси (Taxi Zone) для высадки
- Ссылается на таблицу taxi zones

**`ratecodeid`** - Тип тарифа:
- 1 = Стандартный тариф
- 2 = JFK Airport
- 3 = Newark Airport
- 4 = Nassau or Westchester
- 5 = Negotiated fare
- 6 = Group ride

## Флаги и статусы

**`store_and_fwd_flag`** - Флаг хранения и пересылки данных
- Y = поездка была сохранена в памяти устройства и передана позже
- N = поездка передана в реальном времени

**`payment_type`** - Тип оплаты:
- 1 = Кредитная карта
- 2 = Наличные
- 3 = Без оплаты
- 4 = Спор
- 5 = Неизвестно
- 6 = Voided trip

## Стоимость поездки (в долларах)

**`fare_amount`** - Основная стоимость проезда

**`extra`** - Дополнительные платежи:
- Пиковое время, ночные надбавки и т.д.

**`mta_tax`** - Налог MTA ($0.50)

**`tip_amount`** - Чаевые

**`tolls_amount`** - Плата за проезд по платным дорогам/мостам

**`improvement_surcharge`** - Надбавка за улучшение сервиса ($0.30)

**`total_amount`** - Общая сумма к оплате

**`congestion_surcharge`** - Надбавка за пробки
- Применяется в зонах с высокой загруженностью

**`airport_fee`** - Аэропортовый сбор
- При поездках из аэропортов

## Особенности данных:

1. **Частота обновления**: Ежемесячно
2. **Формат**: Parquet (оптимизирован для аналитики)
3. **Объем**: Обычно несколько миллионов записей в месяц
4. **Качество**: Официальные данные от TLC, но требуют очистки

## Типичные проблемы данных:
- Отрицательные стоимости
- Нулевые расстояния при ненулевой стоимости
- Аномальные значения времени поездки
- Пропущенные значения в некоторых полях

Эти данные широко используются для:
- Анализа транспортных потоков
- Прогнозирования спроса на такси
- Геопространственного анализа
- Анализа доходов и бизнес-метрик

Хочешь, чтобы я помог проанализировать конкретные аспекты этого датасета?

Отличные вопросы! Этот датасет очень богат на возможности для анализа и машинного обучения.

## 📊 **Метрики для анализа**

### **Временные метрики**
```python
# Ежедневная/ежемесячная активность
daily_trips = df.groupBy(date_format("tpep_pickup_datetime", "yyyy-MM-dd")).count()

# Часы пиковой нагрузки
peak_hours = df.groupBy(hour("tpep_pickup_datetime")).count().orderBy("hour")
```

### **Географические метрики**
```python
# Самые популярные зоны посадки/высадки
top_pickup_locations = df.groupBy("pulocationid").count().orderBy(desc("count"))
top_dropoff_locations = df.groupBy("dolocationid").count().orderBy(desc("count"))
```

### **Финансовые метрики**
```python
# Средний чек по часам/дням
avg_fare_by_hour = df.groupBy(hour("tpep_pickup_datetime")).agg(
    avg("total_amount").alias("avg_revenue")
)

# Распределение типов оплат
payment_distribution = df.groupBy("payment_type").count()
```

### **Операционные метрики**
```python
# Средняя длительность и расстояние поездок
df.withColumn("trip_duration_minutes", 
    (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime"))/60
).select(
    avg("trip_duration_minutes").alias("avg_duration"),
    avg("trip_distance").alias("avg_distance")
).show()
```

## 🤖 **Машинное обучение на этом датасете**

### **1. Предсказание длительности поездки**
**Вопрос**: "Сколько времени займет поездка из точки A в точку B?"
```python
# Целевая переменная
df = df.withColumn("trip_duration", 
    unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")
)

# Признаки: день недели, час, зоны, расстояние, пассажиры
```

### **2. Предсказание стоимости поездки**
**Вопрос**: "Сколько будет стоить поездка?"
```python
# Целевая переменная: total_amount
# Признаки: расстояние, время суток, день недели, зоны, пробки
```

### **3. Классификация чаевых**
**Вопрос**: "Даст ли пассажир чаевые?"
```python
# Бинарная классификация
df = df.withColumn("has_tip", when(col("tip_amount") > 0, 1).otherwise(0))
```

### **4. Прогнозирование спроса**
**Вопрос**: "Сколько поездок будет в следующем часу/дне?"
```python
# Временные ряды - агрегация по часам
hourly_demand = df.groupBy(
    hour("tpep_pickup_datetime").alias("hour"),
    dayofweek("tpep_pickup_datetime").alias("day_of_week")
).count()
```

### **5. Кластеризация поездок**
**Вопрос**: "Какие типы поездок существуют?"
```python
# Кластеризация по продолжительности, расстоянию, стоимости
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["trip_distance", "total_amount", "passenger_count"],
    outputCol="features"
)
```

## 💡 **Конкретные бизнес-вопросы для ML**

### **Для такси-компаний:**
- "В какие зоны направить такси в следующий час?"
- "Какие поездки наиболее прибыльные?"
- "Как погода влияет на спрос?"

### **Для пассажиров:**
- "Когда лучше ехать, чтобы избежать пробок?"
- "Сколько в среднем стоит поездка в мой район?"

### **Для городского планирования:**
- "Где построить новые такси-стоянки?"
- "Какие районы нуждаются в улучшении транспорта?"

## 🛠 **Пример пайплайна ML:**
```python
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorAssembler, StringIndexer

# Подготовка признаков
df_ml = df.withColumn("trip_duration", 
    (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime"))/60
).filter(
    (col("trip_duration") > 0) & (col("trip_duration") < 180)  # фильтрация аномалий
)

# Индексация категориальных признаков
zone_indexer = StringIndexer(inputCol="pulocationid", outputCol="pu_zone_index")
hour_indexer = StringIndexer(inputCol=hour("tpep_pickup_datetime"), outputCol="hour_index")

# Векторизация признаков
assembler = VectorAssembler(
    inputCols=["pu_zone_index", "hour_index", "trip_distance", "passenger_count"],
    outputCol="features"
)

# Модель
rf = RandomForestRegressor(featuresCol="features", labelCol="trip_duration")
```

Этот датасет - золотая жила для Data Scientist! Что именно тебя интересует больше - анализ или предсказание?

Отличный вопрос! Кластеризация поездок может раскрыть скрытые паттерны в данных. Давайте разберем подробно.

## 🎯 **Цели кластеризации поездок**

### **Что мы можем узнать:**
- Типичные сценарии поездок
- Аномальные/подозрительные поездки
- Сегменты клиентов
- Оптимизацию бизнес-процессов

## 🔧 **Практическая реализация**

```python
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql.functions import *

# Подготовка данных
df_cluster = df.withColumn("trip_duration_minutes", 
    (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 60
).filter(
    (col("trip_duration_minutes") > 0) & 
    (col("trip_duration_minutes") < 180) &
    (col("trip_distance") > 0) &
    (col("trip_distance") < 50) &
    (col("total_amount") > 0) &
    (col("total_amount") < 200)
)

# Выбор признаков для кластеризации
assembler = VectorAssembler(
    inputCols=["trip_distance", "trip_duration_minutes", "total_amount", "passenger_count"],
    outputCol="raw_features"
)

# Масштабирование признаков (важно для K-means!)
scaler = StandardScaler(
    inputCol="raw_features",
    outputCol="features",
    withStd=True,
    withMean=True
)

# Кластеризация
kmeans = KMeans(
    featuresCol="features",
    predictionCol="cluster",
    k=5,  # количество кластеров
    seed=42
)

# Пайплайн
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[assembler, scaler, kmeans])
model = pipeline.fit(df_cluster)

# Предсказание
result = model.transform(df_cluster)
```

## 📊 **Анализ результатов кластеризации**

### **1. Статистика по кластерам**
```python
# Базовая статистика по кластерам
cluster_stats = result.groupBy("cluster").agg(
    count("*").alias("count"),
    avg("trip_distance").alias("avg_distance"),
    avg("trip_duration_minutes").alias("avg_duration"),
    avg("total_amount").alias("avg_amount"),
    avg("passenger_count").alias("avg_passengers"),
    percentile_approx("total_amount", 0.5).alias("median_amount")
).orderBy("cluster")

cluster_stats.show()
```

### **2. Визуализация центроидов**
```python
# Получаем центры кластеров
centers = model.stages[-1].clusterCenters()

print("Центры кластеров:")
for i, center in enumerate(centers):
    print(f"Кластер {i}: {center}")
```

## 🔍 **Интерпретация кластеров**

### **Типичные паттерны, которые можно обнаружить:**

#### **Кластер 0: "Короткие городские поездки"**
```
- Среднее расстояние: 1-3 мили
- Длительность: 5-15 минут
- Стоимость: $8-15
- Пассажиры: 1-2
```
**Интерпретация**: Быстрые поездки по центру города, возможно, деловые встречи

#### **Кластер 1: "Аэропортовые поездки"**
```
- Среднее расстояние: 15-25 миль  
- Длительность: 30-60 минут
- Стоимость: $50-80
- Пассажиры: 1-2 с багажом
```
**Интерпретация**: Поездки в JFK/LaGuardia

#### **Кластер 2: "Дальние поездки"**
```
- Среднее расстояние: 8-12 миль
- Длительность: 25-40 минут
- Стоимость: $30-50
- Пассажиры: 1-3
```
**Интерпретация**: Поездки между районами, возможно, в пригород

#### **Кластер 3: "Групповые поездки"**
```
- Среднее расстояние: 2-5 миль
- Длительность: 10-20 минут
- Стоимость: $12-25
- Пассажиры: 4-6
```
**Интерпретация**: Групповые поездки, возможно, туристы

#### **Кластер 4: "Ночные поездки"**
```
- Среднее расстояние: 3-7 миль
- Длительность: 15-30 минут  
- Стоимость: $20-40 (с ночными надбавками)
- Пассажиры: 2-3
```
**Интерпретация**: Поездки вечером/ночью

## 💡 **Бизнес-инсайты из кластеризации**

### **Для оптимизации бизнеса:**
```python
# Анализ прибыльности кластеров
profitability = result.groupBy("cluster").agg(
    avg("total_amount").alias("avg_revenue"),
    avg("trip_duration_minutes").alias("avg_duration"),
    (avg("total_amount") / avg("trip_duration_minutes")).alias("revenue_per_minute")
).orderBy(desc("revenue_per_minute"))

profitability.show()
```

### **Что можно узнать:**

1. **Самые прибыльные сегменты** - какие типы поездок приносят больше денег в минуту
2. **Оптимизация флота** - куда направлять такси в разное время суток
3. **Ценообразование** - понимание ценовых сегментов
4. **Аномалии** - выявление мошеннических или ошибочных поездок
5. **Сезонность** - как паттерны меняются по дням недели/времени года

## 🚨 **Обнаружение аномалий**

```python
# Поездки, которые не вписываются в кластеры
anomalies = result.filter(
    (col("trip_distance") > 50) | 
    (col("total_amount") > 200) |
    (col("trip_duration_minutes") > 180)
)

print(f"Обнаружено аномальных поездок: {anomalies.count()}")
```

## 📈 **Визуализация результатов**

```python
# Для визуализации можно преобразовать в pandas
cluster_sample = result.select(
    "trip_distance", "trip_duration_minutes", "total_amount", "cluster"
).sample(0.1).toPandas()

# Scatter plot кластеров
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 8))
sns.scatterplot(data=cluster_sample, x="trip_distance", y="total_amount", hue="cluster", palette="viridis")
plt.title("Кластеризация поездок такси")
plt.xlabel("Расстояние (мили)")
plt.ylabel("Стоимость ($)")
plt.show()
```

## 🎯 **Практическое применение результатов**

**Для такси-компании:**
- "Кластер 1 (аэропорты) - самый прибыльный, увеличить количество машин у аэропортов"
- "Кластер 3 (групповые) - низкая маржинальность, возможно ввести спецтарифы"
- "Кластер 4 (ночные) - высокие надбавки, стимулировать водителей работать ночью"

**Для городского планирования:**
- "Кластер 0 показывает высокий спрос в центре - нужны дополнительные стоянки"
- "Кластер 2 выявляет потребность в улучшении транспорта между районами"

Кластеризация превращает сырые данные в осмысленные бизнес-инсайты!

## 1. Получение данных в слой bronze
В качестве датасета возьмем данные поездок желтого такси. Для этого зафармим файлы отсюда: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

Создадим бакет `bronze` в `MinIO` и напишем функции которые позволят докачивать файлы с учетом уже имеющихся


In [ ]:
!pip install minio

In [ ]:
from minio import Minio
from minio.error import S3Error
import requests
from tqdm import tqdm
import os
import tempfile


# -------------------- Проверить какие файлы фактически существуют на сайте --------------------

def get_available_remote_files(base_url="https://d37ci6vzurychx.cloudfront.net/trip-data",
                               filename_template="yellow_tripdata_{year}-{month:02d}.parquet",
                               year=2024):
    """Проверить какие файлы фактически существуют на сайте"""
    available_files = []
    
    print("🔍 Проверка доступных файлов на сайте...")
    
    for month in tqdm(range(1, 13), desc="Проверка месяца"):
        filename = filename_template.format(year=year, month=month)
        url = f"{base_url}/{filename}"
        
        try:
            # Делаем HEAD запрос чтобы проверить существование файла
            response = requests.head(url, timeout=10)
            if response.status_code == 200:
                available_files.append(filename)
                print(f"  ✓ {filename} - доступен")
            else:
                print(f"  ✗ {filename} - недоступен (код: {response.status_code})")
                
        except requests.exceptions.RequestException as e:
            print(f"  ✗ {filename} - ошибка: {e}")
    
    return available_files

# -------------------- Получить список файлов в MinIO --------------------

def get_local_minio_files(minio_client, 
                          bucket_name="bronze", 
                          prefix="nyc-taxi-data"):
    """Получить список файлов в MinIO"""
    local_files = []
    try:
        objects = minio_client.list_objects(bucket_name, prefix=prefix, recursive=True)
        for obj in objects:
            # Извлекаем имя файла из полного пути
            filename = obj.object_name.replace(f"{prefix}/", "")
            local_files.append(filename)
    except S3Error as e:
        print(f"Ошибка при чтении бакета: {e}")
    
    return local_files

# -------------------- Загрузка только отсутствующих файлов в MinIO --------------------  

def download_missing_files(bucket_name="bronze", 
                           prefix="nyc-taxi-data",
                           base_url="https://d37ci6vzurychx.cloudfront.net/trip-data",
                           filename_template="yellow_tripdata_{year}-{month:02d}.parquet",
                           year=2024):
    """Загрузка только отсутствующих файлов в MinIO"""
    # Настройка клиента MinIO
    minio_client = Minio(
        "minio:9000",
        access_key="minioadmin",
        secret_key="minioadmin",
        secure=False
    )
    
    # Создаем бакет если нужно
    try:
        if not minio_client.bucket_exists(bucket_name):
            minio_client.make_bucket(bucket_name)
            print(f"✓ Бакет {bucket_name} создан")
    except S3Error as e:
        return [f"✗ Ошибка бакета: {e}"]
    
    # Получаем списки файлов
    remote_files = get_available_remote_files(base_url, filename_template, year)
    local_files = get_local_minio_files(minio_client, bucket_name, prefix)
    
    # Находим отсутствующие файлы
    missing_files = list(set(remote_files) - set(local_files))

    # Блок статистики
    print(f"\n📊 СТАТИСТИКА:")

    print(f"• Загружено в MinIO: {len(local_files)} файл(ов)")    

    print()    
    print(f"• Доступно на сайте: {len(remote_files)} файл(ов)")
    for file in sorted(remote_files):
        print(f"     - {file}")

    print()
    if missing_files:
        print(f"• Из них отсутствует в MinIO: {len(missing_files)} файл(ов)")
        for file in sorted(missing_files):
            print(f"     - {file}")
    
    if not missing_files:
        return ["✅ Все доступные файлы уже загружены"]
    
    results = []
    
    # Скачиваем только отсутствующие файлы
    for filename in tqdm(missing_files, desc="Загрузка недостающих"):
        url = f"{base_url}/{filename}"
        
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()
            
            # Создаем временный файл
            with tempfile.NamedTemporaryFile(delete=False, suffix='.parquet') as temp_file:
                temp_path = temp_file.name
                
                # Скачиваем файл на диск
                total_size = int(response.headers.get('content-length', 0))
                for chunk in response.iter_content(chunk_size=8192 * 8):
                    if chunk:
                        temp_file.write(chunk)
            
            # Получаем реальный размер файла
            file_size = os.path.getsize(temp_path)
            
            # Загружаем в MinIO
            minio_client.fput_object(
                bucket_name=bucket_name,
                object_name=f"{prefix}/{filename}",
                file_path=temp_path
            )
            
            # Удаляем временный файл
            os.unlink(temp_path)
            
            results.append(f"✓ {filename} ({file_size / (1024 * 1024):.1f} MB)")
            
        except Exception as e:
            # Удаляем временный файл в случае ошибки
            if 'temp_path' in locals():
                try:
                    os.unlink(temp_path)
                except:
                    pass
            results.append(f"✗ {filename}: {e}")
    
    return results

# --------- Сравниваем список доступного в MinIO и на сайте на указанный год ---------

def show_current_state(bucket_name="bronze", 
                       prefix="nyc-taxi-data",
                       base_url="https://d37ci6vzurychx.cloudfront.net/trip-data",
                       filename_template="yellow_tripdata_{year}-{month:02d}.parquet",
                       year=2024):
    """Показать текущее состояние файлов"""
    minio_client = Minio(
        "minio:9000",
        access_key="minioadmin", 
        secret_key="minioadmin",
        secure=False
    )
    
    remote_files = get_available_remote_files(base_url, filename_template, year)
    local_files = get_local_minio_files(minio_client, bucket_name, prefix)
    missing_files = list(set(remote_files) - set(local_files))
    
    print("\n📊 ТЕКУЩЕЕ СОСТОЯНИЕ:")
    print(f"• Загружено в MinIO: {len(local_files)} файл(ов)")

    print()
    print(f"• Доступно на сайте: {len(remote_files)} файл(ов)")
    for file in sorted(remote_files):
        print(f"     - {file}")    

    print()
    print(f"• Из них отсутствует в MinIO: {len(missing_files)} файл(ов)")
    if missing_files:
        for file in sorted(missing_files):
            print(f"     - {file}")

In [ ]:
# Передаем параметры источника и получателя
results = download_missing_files(
    bucket_name="bronze", 
    prefix="nyc-taxi-data",
    base_url="https://d37ci6vzurychx.cloudfront.net/trip-data",
    filename_template="yellow_tripdata_{year}-{month:02d}.parquet",
    year=2025
)

print("Результаты дозагрузки:")
for result in sorted(results):
    print(result)

In [ ]:
# Сравниваем список доступного в MinIO и на сайте на указанный год 
show_current_state(bucket_name="bronze", 
                   prefix="nyc-taxi-data",
                   base_url="https://d37ci6vzurychx.cloudfront.net/trip-data",
                   filename_template="yellow_tripdata_{year}-{month:02d}.parquet",
                   year=2025)

# Обработка данных

Данные за разные года приходят с разной структурой.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

drivers = [
    "/home/jovyan/work/spark-jars/hadoop-aws-3.3.4.jar",             # S3
    "/home/jovyan/work/spark-jars/aws-java-sdk-bundle-1.12.262.jar", # S3
    "/home/jovyan/work/spark-jars/wildfly-openssl-1.0.7.Final.jar",  # S3
    "/home/jovyan/work/spark-jars/postgresql-42.6.0.jar",            # PostgreSQL
]

spark = (SparkSession.builder
         .appName("mustdayker-Spark")
         .master("spark://spark-master:7077") 
         .config("spark.jars", ",".join(drivers))
         .getOrCreate()
        )

25/11/04 15:43:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/04 15:43:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
df_2022 = spark.read.format("parquet").load("s3a://bronze/nyc-taxi-data/yellow_tripdata_2022-01.parquet")
df_2024 = spark.read.format("parquet").load("s3a://bronze/nyc-taxi-data/yellow_tripdata_2024-01.parquet")

In [ ]:
df_2022.select("VendorID", "passenger_count", "trip_distance", "RatecodeID", "PULocationID").printSchema()
df_2024.select("VendorID", "passenger_count", "trip_distance", "RatecodeID", "PULocationID").printSchema()

# Нормализация типов, загрузка в `silver`

Для того чтобы можно было загружать данные как единый массив напишем функции нормализации.

Функции проверяют уже обработанные срезы, сравнивают с новыми которые появились в Bronze и обрабатывают только новые срехы

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, IntegerType
import re
from minio import Minio
from minio.error import S3Error
import time

# Конфигурация MinIO
MINIO_ENDPOINT = 'minio:9000'  # замените на ваш endpoint
MINIO_ACCESS_KEY = 'minioadmin'
MINIO_SECRET_KEY = 'minioadmin'
MINIO_BUCKET_BRONZE = 'bronze'
MINIO_BUCKET_SILVER = 'silver'

def get_minio_client():
    """Создает и возвращает клиент MinIO"""
    return Minio(
        MINIO_ENDPOINT,
        access_key=MINIO_ACCESS_KEY,
        secret_key=MINIO_SECRET_KEY,
        secure=False  # используйте True для HTTPS
    )

def extract_month_from_filename(file_path):
    """Извлекает месяц из имени файла в формате YYYY-MM"""
    match = re.search(r'(\d{4}-\d{2})', file_path)
    return match.group(1) if match else None

def get_processed_slices():
    """Возвращает список уже обработанных срезов из Silver используя MinIO"""
    try:
        client = get_minio_client()
        processed_slices = set()
        
        # Получаем список объектов в префиксе nyc-taxi-data/
        objects = client.list_objects(MINIO_BUCKET_SILVER, prefix="nyc-taxi-data/", recursive=True)
        
        for obj in objects:
            # Извлекаем месяц из пути
            month = extract_month_from_filename(obj.object_name)
            if month:
                processed_slices.add(month)
        
        print(f"📁 Найдено обработанных срезов в Silver: {len(processed_slices)}")
        return processed_slices
        
    except S3Error as e:
        if e.code == 'NoSuchBucket':
            print("⚠️ Бакет Silver не существует или пустой")
        else:
            print(f"⚠️ Ошибка при чтении Silver слоя: {e}")
        return set()
    except Exception as e:
        print(f"⚠️ Не удалось прочитать Silver слой: {e}")
        return set()

def get_bronze_files_with_months():
    """Возвращает список файлов из Bronze с извлеченными месяцами используя MinIO"""
    try:
        client = get_minio_client()
        bronze_files = []
        
        # Получаем список объектов в префиксе nyc-taxi-data/
        objects = client.list_objects(MINIO_BUCKET_BRONZE, prefix="nyc-taxi-data/", recursive=True)
        
        for obj in objects:
            if obj.object_name.endswith('.parquet'):
                month = extract_month_from_filename(obj.object_name)
                if month:
                    # Формируем S3 путь для Spark
                    s3_path = f"s3a://{MINIO_BUCKET_BRONZE}/{obj.object_name}"
                    bronze_files.append({
                        'path': s3_path,
                        'month': month,
                        'file_name': obj.object_name.split('/')[-1]
                    })
        
        print(f"📁 Найдено файлов в Bronze: {len(bronze_files)}")
        return bronze_files
        
    except Exception as e:
        print(f"❌ Ошибка при чтении Bronze слоя: {e}")
        return []

def standardize_nyc_taxi_data(input_path, output_path):
    """
    Стандартизирует данные NYC Taxi (ваша существующая функция без изменений)
    """
    output_path = output_path.replace('.parquet', '')
    
    # Читаем исходные данные
    df = spark.read.parquet(input_path)
    
    # 1. Приводим все названия колонок к нижнему регистру
    for col_name in df.columns:
        df = df.withColumnRenamed(col_name, col_name.lower())
    
    # 2. Определяем маппинг типов для унификации
    type_mapping = {
        "vendorid": IntegerType(),
        "pulocationid": IntegerType(), 
        "dolocationid": IntegerType(),
        "payment_type": IntegerType(),
        "ratecodeid": IntegerType(),
        "passenger_count": IntegerType(),
        "fare_amount": DoubleType(),
        "extra": DoubleType(),
        "mta_tax": DoubleType(),
        "tip_amount": DoubleType(),
        "tolls_amount": DoubleType(),
        "improvement_surcharge": DoubleType(),
        "total_amount": DoubleType(),
        "congestion_surcharge": DoubleType(),
        "airport_fee": DoubleType(),
        "cbd_congestion_fee": DoubleType(),
        "trip_distance": DoubleType()
    }
    
    # 3. Применяем преобразование типов с обработкой ошибок
    for col_name, target_type in type_mapping.items():
        if col_name in df.columns:
            df = df.withColumn(
                col_name, 
                F.coalesce(
                    F.col(col_name).cast(target_type), 
                    F.lit(0 if target_type == IntegerType() else 0.0)
                )
            )
    
    # 4. Гарантируем порядок колонок для consistency
    expected_columns = [
        "vendorid", "tpep_pickup_datetime", "tpep_dropoff_datetime",
        "passenger_count", "trip_distance", "ratecodeid", "store_and_fwd_flag",
        "pulocationid", "dolocationid", "payment_type", "fare_amount", "extra",
        "mta_tax", "tip_amount", "tolls_amount", "improvement_surcharge",
        "total_amount", "congestion_surcharge", "airport_fee", "cbd_congestion_fee"
    ]
    
    # Выбираем только существующие колонки в правильном порядке
    final_columns = [col for col in expected_columns if col in df.columns]
    df_standardized = df.select(final_columns)
    
    # 5. Сохраняем с оптимальными настройками
    (df_standardized
     .coalesce(1)
     .write
     .mode("overwrite")
     .option("compression", "snappy")
     .parquet(output_path)
    )
    
    print(f"✅ Стандартизировано: {input_path} -> {output_path}")
    return df_standardized

def process_incremental_nyc_taxi_files():
    """Обрабатывает только новые файлы NYC Taxi из Bronze в Silver"""
    
    # Получаем списки обработанных и доступных файлов через MinIO
    processed_slices = get_processed_slices()
    bronze_files = get_bronze_files_with_months()
    
    # Фильтруем только новые файлы
    new_files = [f for f in bronze_files if f['month'] not in processed_slices]
    
    print(f"📊 Статистика:")
    print(f"   - Всего в Bronze: {len(bronze_files)}")
    print(f"   - Уже в Silver: {len(processed_slices)}") 
    print(f"   - Новых для обработки: {len(new_files)}")
    
    if not new_files:
        print("🎉 Все срезы уже обработаны! Ничего делать не нужно.")
        return
    
    # Обрабатываем только новые файлы
    for i, file_info in enumerate(new_files, 1):
        input_path = file_info['path']
        file_name = file_info['file_name']
        output_path = f"s3a://{MINIO_BUCKET_SILVER}/nyc-taxi-data/{file_name.replace('.parquet', '')}"
        
        print(f"🔄 Обрабатываю новый срез ({i}/{len(new_files)}): {file_info['month']}")
        
        try:
            standardize_nyc_taxi_data(input_path, output_path)
            print(f"✅ Успешно обработан: {file_info['month']}")
        except Exception as e:
            print(f"❌ Ошибка при обработке {file_info['month']}: {e}")
    
    print(f"🎉 Обработка завершена! Обработано {len(new_files)} новых срезов.")

# Дополнительная функция для отладки - просмотр структуры бакетов
def inspect_buckets():
    """Функция для отладки - показывает структуру бакетов"""
    try:
        client = get_minio_client()
        
        print("🔍 Инспекция Bronze бакета:")
        bronze_objects = client.list_objects(MINIO_BUCKET_BRONZE, prefix="nyc-taxi-data/", recursive=True)
        for obj in bronze_objects:
            print(f"   Bronze: {obj.object_name}")
        
        print("🔍 Инспекция Silver бакета:")
        silver_objects = client.list_objects(MINIO_BUCKET_SILVER, prefix="nyc-taxi-data/", recursive=True)
        for obj in silver_objects:
            print(f"   Silver: {obj.object_name}")
            
    except Exception as e:
        print(f"❌ Ошибка при инспекции бакетов: {e}")

In [ ]:
# Для отладки сначала посмотрим структуру бакетов
# inspect_buckets()

start_time = time.time()

# Запускаем инкрементальную обработку
process_incremental_nyc_taxi_files()

end_time = time.time()
execution_time = end_time - start_time
print(f"Время выполнения: {execution_time:.4f} секунд")

# Работа со слоем Silver

In [32]:
df = (spark.read
      .format("parquet")
      .load("s3a://silver/nyc-taxi-data/yellow_tripdata_2022-*"))

In [33]:
df.count()

39656098

In [25]:
from pyspark.sql.functions import year, date_format, count, col

In [28]:
# Если поле уже имеет тип timestamp/date
result = (df
         .filter(year("tpep_pickup_datetime") == 2025)          
         .withColumn("year", year("tpep_pickup_datetime"))
         .withColumn("month", date_format("tpep_pickup_datetime", "MM"))  # "MM" вернет 01, 02, 03...
         .groupBy("year", "month")
         .agg(count("*").alias("record_count"))
         .orderBy("year", "month"))

In [29]:
result.show(100)

[Stage 36:=====>                                                   (1 + 9) / 10]

+----+-----+------------+
|year|month|record_count|
+----+-----+------------+
|2025|   01|     3475234|
|2025|   02|     3577542|
|2025|   03|     4145229|
|2025|   04|     3970568|
|2025|   05|     4591844|
|2025|   06|     4322949|
|2025|   07|     3898971|
|2025|   08|     3574080|
|2025|   09|     4251009|
|2025|   10|           1|
+----+-----+------------+



In [30]:
from pyspark.sql.functions import col, sum, count

In [34]:
from pyspark.sql.functions import col, sum

# Считаем пропущенные значения
null_counts = df.agg(*[
    sum(col(c).isNull().cast("int")).alias(c) 
    for c in df.columns
])

# Преобразуем в pandas для красивого отображения
null_counts_pd = null_counts.toPandas().T
null_counts_pd.columns = ['null_count']
null_counts_pd['total_count'] = df.count()
null_counts_pd['null_pct'] = (null_counts_pd['null_count'] / null_counts_pd['total_count'] * 100).round(2)

print(null_counts_pd)

[Stage 49:==============================================>         (10 + 2) / 12]

                       null_count  total_count  null_pct
vendorid                        0     39656098      0.00
tpep_pickup_datetime            0     39656098      0.00
tpep_dropoff_datetime           0     39656098      0.00
passenger_count                 0     39656098      0.00
trip_distance                   0     39656098      0.00
ratecodeid                      0     39656098      0.00
store_and_fwd_flag        1368303     39656098      3.45
pulocationid                    0     39656098      0.00
dolocationid                    0     39656098      0.00
payment_type                    0     39656098      0.00
fare_amount                     0     39656098      0.00
extra                           0     39656098      0.00
mta_tax                         0     39656098      0.00
tip_amount                      0     39656098      0.00
tolls_amount                    0     39656098      0.00
improvement_surcharge           0     39656098      0.00
total_amount                   